{ Instruction<sub>n-2</sub>, Instruction<sub>n-1</sub>, Instruction<sub>n</sub> ; ImageCaption<sub>n</sub> }

In [3]:
import json
from tqdm import tqdm

RECIPES_PATH = ''
CAPTIONS_FILE = ''
CAPTION_TYPE = "current_step"

RESULT_FILE = f'c0_{CAPTION_TYPE}_caption_dataset.json'

WINDOW = 1

with open(RECIPES_PATH, 'r') as f:
    recipes = json.load(f)

with open(CAPTIONS_FILE, 'r') as f:
    captions = json.load(f)


print(len(captions)*4)

results = []

recipe_items = recipes.items()

key_errors = 0

for recipe_id, _recipe in tqdm(recipe_items):
    try:
        recipe = _recipe['recipe']

        context_arr = []
        steps = recipe['instructions']
        for i, step in enumerate(steps):
            if len(context_arr) < WINDOW:
                context_arr.append(f"Step {step['stepNumber']}: {step['stepText']}\n")
            else:
                #context_arr[0] = f"Step 1: {captions[f'{recipe_id}_{i-2+1}'][CAPTION_TYPE]}\n"
                #context_arr[1] = f"Step 2: {steps[i-1]['stepText']}\n"
                #context_arr[2] = f"Step 3: {step['stepText']}\n"
                
                #context_arr[0] = f"Step 1: {captions[f'{recipe_id}_{i-1+1}'][CAPTION_TYPE]}\n"
                #context_arr[1] = f"Step 2: {step['stepText']}\n"

                context_arr[0] = f"Step 1: {step['stepText']}\n"

            image_id = f"{recipe_id}_{step['stepNumber']}"
            caption = captions[image_id][CAPTION_TYPE]

            context = "".join(context_arr)

            sample = {
                "image": image_id,
                "steps": context,
                "caption": caption,
                "recipe_id": recipe_id,
                "step": i
            }

            results.append(sample) 
    except KeyError as ke:
        # print("KeyError", ke)
        key_errors += 1
        continue

print(f"Got {key_errors} key errors.")

with open(RESULT_FILE, "w") as f:
    json.dump(results, f, indent=4)

TypeError: can only concatenate str (not "int") to str

Alpaca format

In [7]:
import json
import random
from sklearn.model_selection import train_test_split

CAPTION_TYPE = "c0"

INPUT = f'.json'
ALPACA_DATA = ".json"

INSTRUCTION = "Give me the image caption for this recipe."

dataset = []

with open(INPUT, "r") as f:
    input_dataset = json.load(f)

for sample in input_dataset:
    sample_alpaca = {
        "instruction": INSTRUCTION,
        "input": sample["steps"],
        "output": sample["caption"],
        "recipe_id": sample["recipe_id"],
        "step": sample["step"]
    }

    dataset.append(sample_alpaca)

random.seed(10)
random.shuffle(dataset)

train, eval = train_test_split(dataset, test_size=0.2)

with open(ALPACA_DATA.format(CAPTION_TYPE, CAPTION_TYPE, "train"), "w") as f:
    json.dump(train, f, indent=4)

if eval:
    with open(ALPACA_DATA.format(CAPTION_TYPE, CAPTION_TYPE, "eval"), "w") as f:
        json.dump(eval, f, indent=4)

print(f"Dataset has {len(dataset)} examples.")

Dataset has 5562 examples.


In [10]:
ALPACA_DATA_TRAIN = '.json'
ALPACA_DATA_VAL = '.json'

NUM_EXAMPLES=5000

with open(ALPACA_DATA_TRAIN, "w") as f:
    json.dump(dataset[:NUM_EXAMPLES-500], f, indent=4)

with open(ALPACA_DATA_VAL, "w") as f:
    json.dump(dataset[NUM_EXAMPLES-500:], f, indent=4)
